This is a simple demo of the "ETC" mode for MOSAIC. To use this you need the `oc/mosaic` branch of Scopesim and the `oc/mosaic` branch of the irdb. The implementation creates one-dimensional spectra for the output (summed over the fibres in a bundle for the MOS modes). The first sections demonstrate how to use Scopesim for the MOS and mIFU modes. The final section provides a more detailed description of the implementation and instrument definition that is currently used.

In [ ]:
import scopesim as sim

Change the path below to your local irdb copy.

In [ ]:
sim.link_irdb("/path/to/your/irdb")

## Instrument modes

In [ ]:
cmd = sim.UserCommands(use_instrument="MOSAIC")

In [ ]:
cmd.modes

- The visual modes map spectra to a pseudo-detector of length 13000 pixels; the gap between the two 6k detectors in the real instrument is not simulated.
- The near-infrared modes map spectra to a 4k x 4k detector.
- As stated above, the MOS modes collapse the 7 (low-res modes) or 19 (high-res modes) fibres in a bundle to a single one-dimensional spectrum, the output is a FITS binary table with wavelength and flux (in ADU).
- The mIFU modes return a 4k x 4k image with each of the 4xx fibre spectra mapped onto a detector row each. Rearrangement of the spectra into a cube has not been implemented.

## MOS modes
All the MOS modes work identically, and we will only demonstrate the low-resolution R band mode.

In [ ]:
cmd = sim.UserCommands(use_instrument="MOSAIC", set_modes=['MOS-LR-R'])

In [ ]:
mosaic = sim.OpticalTrain(cmd)

The following gives an overview of the effects that are included in the optical train. 

In [ ]:
mosaic.effects

We start by observing blank sky; this will be useful for background subtraction when we observe a source later.

In [ ]:
mosaic.observe()

In [ ]:
sky = mosaic.readout(exptime=600)[0]

The result is a binary table with columns `wavelength` and `spectrum`:

In [ ]:
sky[1].data

In [ ]:
from matplotlib import pyplot as plt
plt.plot(sky[1].data['wavelength'], sky[1].data['spectrum'])

As a source, we'll use a star of 15 mag:

In [ ]:
from astropy import units as u

In [ ]:
src = sim.source.source_templates.star(flux=15 * u.mag)

In [ ]:
mosaic.observe(src)

In [ ]:
star = mosaic.readout(exptime=600)[0]      # rather generous exposure time...

In [ ]:
plt.plot(star[1].data['wavelength'], star[1].data['spectrum'], label="raw spectrum")
plt.plot(star[1].data['wavelength'], star[1].data['spectrum'] - sky[1].data['spectrum'], label="background subtracted")
plt.legend()
plt.xlabel("Wavelength [um]")
plt.ylabel("Electrons");

Strictly speaking, the flux units is ADU, but the gain is currently set to 1, so that 1 ADU corresponds to 1 electron collected over the exposure time (600 s).
The spectrum is the sum over the seven fibres (19 for the high-resolution modes), each covering a solid angle of 0.0118 arcsec$^2$. 

Note that the `mosaic.readout()` method above has been set by the `exptime` keyword, and the exposure is a single readout of that integration time. This is appropriate for the visual modes. The near-infrared modes can also receive a simple `exptime`. In this case, the exposure time is automatically split into `ndit` subexposures of integration time `dit` each (cf. the readout of the mIFU example below). It is also possible to set `dit` and `ndit` directly. 

## mIFU modes

In [ ]:
cmd = sim.UserCommands(use_instrument="MOSAIC", set_modes=["mIFU-LR-J"])

In [ ]:
mosaic = sim.OpticalTrain(cmd)

In [ ]:
mosaic.observe()

In [ ]:
hdul = mosaic.readout(exptime=3600)[0]

In [ ]:
plt.imshow(hdul[1].data[900:1100, 1900:2100], norm='log', origin='lower')
plt.colorbar();

In [ ]:
plt.plot(hdul[1].data[991, :])

The mIFU mode is incomplete - it is currently not possible to deduce the wavelength vector and the spatial arrangement of the fibres from the readout alone. A table format might be better suited for that purpose.

## Instrument package
In this section we describe how the instrument is defined for Scopesim and what data files and parameters are used. The instrument configuration is found in `irdb/MOSAIC` (a release package will be provided once the code and configuration has been validated). The main file is `default.yaml` where in particular the available instrument modes are defined (see above for a list of these modes). The `OpticalTrain`, i.e. the software representation of the entire system of atmosphere, telescope, instrument optics and detector, is built as a series of `Effect` objects, which are set along with relevant parameters in further `yaml` files, as appropriate for each instrument mode. 
All modes use the additional `Armazones` and `ELT` packages:
- `irdb/Armazones/Armazones.yaml` sets a single effect, `skycalc_atmosphere`, which provides atmospheric transmission and emission spectra as taken from ESO's skycalc server. The two parameters `spectral_resolution` and `spectral_bin_width` have been carefully chosen for each mode and should not be changed (unless for experimental purposes).
- `irdb/ELT/ELT.yaml` defines `telescope_reflection`, which provides reflectivity and thermal emission from the telescope mirrors. The parameter `!TEL.ter_curve.filename` is by default set to `TER_ELT_5_mirror.dat`. The irdb contains files for 5- and 6-mirror configurations (the latter including a flat mirror to deflect light to a side port), the latter with exposed (field-tracking) or hidden (pupil-tracking) telescope spiders, and for clean and not so clean mirror segments.

<img src="ELT_emissivity.png" style="width:400px"> </img>
<img src="ELT_throughput.png" style="width:400px"> </img>

The instrument is split into a visual and a near-infrared arm, described by `MOSAIC_VIS.yaml` and `MOSAIC_NIR.yaml`, respectively. 
- The transmission of the visual arm is given as a single combined `system transmission` effect, which uses the file named in the parameter "!OBS.ter_file" (TER = transmission, emission, reflection). For the near-infrared arm, the transmission is split into `fibre transmission` and `spectrograph transmission` (parameters `!OBS.fibre_ter_file` and `!OBS.spec_ter_file`). The fibre transmission files are named `TER_mos_*` to distinguish them from the IFU fibres, `TER_mIFU_*`. The available data are rather crude at present.

<img src="MOS_VIS_transmission.png" style="width:400px"> </img>
<img src="MOS_NIR_transmission.png" style="width:400px"> </img>

- The point-spread function (effect `psf`) is currently taken to be a `SeeingPSF`, with FWHM settable via `!OBS.psf_fwhm`. This parameter is currently set to a default of 0.2 arcsec for all modes, probably not always a good value.
- The line-spread function (effect `lsf`) is a convolution of a top-hat of width 4.5 spectral bins in the visual and 2.75 spectral bins in the near-infrared (the effect parameter `lsfwidth` is not currently user-settable, except by editing the yaml file; this should change in the future) with a Gaussian with standard deviation of 1 spectral bin. A spectral bin is given by the dispersion (um per pixel) on the detector.
- The `fibre_bundle` effect determines the outline of the fibre bundle on the sky, and the `spectral_traces` effect is responsible for mapping the spectra into the two-dimensional `ImagePlane` (i.e. the detector focal plane). Both effects use the same trace file. Trace files are available for all instrument modes as multi-extension FITS files. The spatial arrangement of the fibres in a bundle is given as a table in the second extension of the file. Note that the fibre apertures are modelled as squares rather than hexagons (this makes it easier to sample the source cube); of course the squares cover the same solid angle on sky as the hexagons.

<img src="MOS-LR-fibre_bundle.png" style="width:400px"> </img>
<img src="MOS-HR-fibre_bundle.png" style="width:400px"> </img>

The remaining traces map the spectra into the detector. This is done in a strictly linear manner, i.e. with constant dispersion (different for each mode). Each fibre is mapped as a one-dimensional spectrum onto a single row of the `ImagePlane`. 

The `ImagePlane` is produced by the `mosaic.observe()` command and is an image of the expected flux in photon/s in the detector plane, without noise. It is accessible by `mosaic.image_plane.hdu`.

The `mosaic.readout()` command subsequently invokes the detector configuration to create a noisy detector readout from the `ImagePlane`. The detectors are defined in `MOSAIC_DET_VIS.yaml` and `MOSAIC_DET_NIR.yaml`, respectively. The geometric layout of the detectors is given by the effect `detector_array`, which reads `FPA_mosaic_VIS_layout.dat` and `FPA_mosaic_NIR_layout.dat`, respectively. The visual detector is currently described as a single 13000 x 160 array rather than the four 6k x 6k detectors foreseen for the actual instrument (hence the "s(i)mpl(e)" in the yaml file name) . This is because Scopesim only simulates a single fibre bundle, so a narrow effective detector is sufficient. Masking the gap between the two detectors is left to the user. The NIR detector is modeled as a single 4k x 4k detector (as in the actual instrument) to have enough room for the mIFU mode. 

The QE is handled by the effect `quantum_efficiency`, which uses the files `QE_VIS_ML2_depleted.dat` and `QE_detector_NIR.dat`, respectively.

<img src="MOSAIC-QE.png" style="width:400px"> </img>

The visual detector currently only applies shot noise; dark current and read noise still need to be included. The near-infrared detector includes these effects; the parameters are taken from our experience with MICADO. 

Finally, both detector yamls include the effect `collapse_1d`, which sums up the separate spectra from the fibres and produces a single output spectrum as described above. It is possible to switch this off with `mosaic['collapse_1d'].include=False` to see the "detector image". However, due to the way the spectra are currently formed, this is probably not of much interest.